In [ ]:
import pandas as pd
import pickle
from string import ascii_uppercase as alphabet
import json

In [ ]:
all_tables = pd.read_html("https://en.wikipedia.org/wiki/UEFA_Euro_2024")

In [ ]:
dict_table = {}

for letter, i in zip(alphabet, range(18, 57, 7)):
    print(f"Group {letter}")
    df = all_tables[i]
    df = df.rename(columns={df.columns[1]:'Team'})
    df = df.drop("Qualification", axis=1)
    dict_table[f"Group {letter}"] = df

In [ ]:
dict_table["Group A"]

In [ ]:
with open ("_data/dict_table", "wb") as output:
    pickle.dump(dict_table, output)

In [ ]:
from bs4 import BeautifulSoup
import requests

# Generate list of years from 1960 to 2020 in 4-year increments
years = list(range(1960, 2021, 4))

def get_matches(year):
    web = f'https://en.wikipedia.org/wiki/UEFA_Euro_{year}'
    response = requests.get(web)
    content = response.text
    soup = BeautifulSoup(content, "lxml")
    matches = soup.find_all("div", class_="footballbox")

    home = []
    score = []
    away = []

    for match in matches:
        home.append(match.find('th', class_='fhome').get_text())
        score.append(match.find('th', class_='fscore').get_text())
        away.append(match.find('th', class_='faway').get_text())

    dict_football = {"home": home, "score": score, "away": away}
    df_football = pd.DataFrame(dict_football)
    df_football["year"] = year
    return df_football


In [ ]:
# Extracting historic Euros scores
Euros = [get_matches(year) for year in years]
df_Euros = pd.concat(Euros, ignore_index=True)
df_Euros.to_csv("_data/Euros_historical_data.csv", index=False)

In [ ]:
# Extracting this years fixtures
df_fixture = get_matches(2024)
df_fixture["Match_no"] = df_fixture['score'].str.extract('(\d+)').astype(int)
df_fixture.sort_values("Match_no").to_csv("_data/Euros_fixtures.csv", index=False)

In [ ]:
df_historical = pd.read_csv("_data/Euros_historical_data.csv")
df_fixture = pd.read_csv("_data/Euros_fixtures.csv")

In [ ]:
df_fixture["home"] = df_fixture["home"].str.strip()
df_fixture["away"] = df_fixture["away"].str.strip()

In [ ]:
df_historical_data = (
    df_historical
    .drop_duplicates()
    .sort_values("year")
)

In [ ]:
# Clean score, home/away cols
df_historical_data["score"] = df_historical_data["score"].str.replace("[^\d\–]","", regex=True)
df_historical_data["home"] = df_historical_data["home"].str.strip()
df_historical_data["away"] = df_historical_data["away"].str.strip()

# Splitting score into home & away cols
df_historical_data[["HomeGoals", "AwayGoals"]] = df_historical_data["score"].str.split("–", expand=True)
df_historical_data.drop("score", axis=1, inplace=True)

# Renaming cols and changing data types
df_historical_data.rename(columns={"home": "HomeTeam", "away": "AwayTeam",
                                   "year": "Year"}, inplace=True)
df_historical_data = df_historical_data.astype({"HomeGoals": int, "AwayGoals": int,
                                                "Year": int})

# Creating new col "totalgoals"
df_historical_data["TotalGoals"] = df_historical_data["HomeGoals"] + df_historical_data["AwayGoals"]

In [ ]:
df_historical_data.to_csv("_data/clean_Euros_matches.csv", index=False)
df_fixture.to_csv("_data/clean_Euros_fixtures.csv", index=False)

In [ ]:
dict_table = pickle.load(open("_data/dict_table", "rb"))
df_historical_clean = pd.read_csv("_data/clean_Euros_matches.csv")
df_fixture_clean = pd.read_csv("_data/clean_Euros_fixtures.csv")

In [ ]:
# Replace "Germany" with "Germany (H)" in both cols
df_updated = df_historical_data.copy() # Copy to avoid overwriting original df
df_updated.loc[df_updated["HomeTeam"] == "Germany", "HomeTeam"] = "Germany (H)"
df_updated.loc[df_updated["AwayTeam"] == "Germany", "AwayTeam"] = "Germany (H)"

df_historical_clean = df_updated

In [ ]:
df_home = df_historical_clean[["HomeTeam", "HomeGoals", "AwayGoals"]]
df_away = df_historical_clean[["AwayTeam", "HomeGoals", "AwayGoals"]]

df_home = df_home.rename(columns={"HomeTeam":"Team", "HomeGoals":"GoalsScored",
                                  "AwayGoals":"GoalsConceded"})
df_away = df_away.rename(columns={"AwayTeam":"Team", "HomeGoals":"GoalsConceded",
                                  "AwayGoals":"GoalsScored"})

df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby(["Team"]).mean()
df_team_strength

In [ ]:
# Replace with "Germany (H)"
df_fixture.replace({"home": "Germany"}, {"home": "Germany (H)"}, inplace=True)
df_fixture.replace({"away": "Germany"}, {"away": "Germany (H)"}, inplace=True)

In [ ]:
from scipy.stats import poisson

# Function to predict the points gained from a match for each team
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home, 'GoalsScored'] * df_team_strength.at[away, "GoalsConceded"]
        lamb_away = df_team_strength.at[away, 'GoalsScored'] * df_team_strength.at[home, "GoalsConceded"]

        prob_home, prob_away, prob_draw = 0, 0, 0

        for x in range(0, 11):
            for y in range(0, 11):
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)

                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                elif x < y:
                    prob_away += p
    
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw

        return(points_home, points_away)
    
    else:
        return(0,0)

In [ ]:
print(predict_points("Germany (H)", "Scotland"))

In [ ]:
# Setting out the fixture rounds
df_fixture_group44 = df_fixture[:36].copy()
df_fixture_R16 = df_fixture[36:44].copy()
df_fixture_quarter = df_fixture[44:48].copy()
df_fixture_semi = df_fixture[48:50].copy()
df_fixture_final = df_fixture[50:].copy()

In [ ]:
for group in dict_table:
    teams_in_group = dict_table[group]["Team"].values
    df_fixture_group6 = df_fixture_group44[df_fixture_group44["home"].isin(teams_in_group)]

    for index, row in  df_fixture_group6.iterrows():
        home, away = row["home"], row["away"]
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]["Team"] == home, "Pts"] += int(points_home)
        dict_table[group].loc[dict_table[group]["Team"] == away, "Pts"] += int(points_away)

    dict_table[group] = dict_table[group].sort_values("Pts", ascending = False).reset_index()
    dict_table[group] = dict_table[group][["Team", "Pts"]]
    dict_table[group] = dict_table[group].round(0)


In [ ]:
dict_table["Group B"].iloc[2]["Pts"]

In [ ]:
import pandas as pd

# Create an empty DataFrame for ranking of 3rd place teams
third_placed_teams = pd.DataFrame(columns=["Pos", "Grp", "Team", "Pld",
                                           "W", "D", "L", "GF", "GA", "GD", "Pts", "Qualification"]).reset_index(drop=True)

# Iterate over each group in dict_table
for i, group in enumerate(dict_table, start=1):
    # Calculate points for teams in the group based on predictions
    teams_in_group = dict_table[group]["Team"].values

    for index, row in df_fixture_group6.iterrows():
        home, away = row["home"], row["away"]
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]["Team"] == home, "Pts"] += points_home
        dict_table[group].loc[dict_table[group]["Team"] == away, "Pts"] += points_away

    # Sort teams in the group based on points
    dict_table[group] = dict_table[group].sort_values("Pts", ascending=False).reset_index(drop=True)
    dict_table[group] = dict_table[group][["Team", "Pts"]]
    dict_table[group] = dict_table[group].round(0)

    # Add third-placed team to the ranking DataFrame
    third_place_team = dict_table[group].iloc[2]
    add_third_place = pd.DataFrame([{
        "Pos": i,
        "Grp": group,
        "Team": third_place_team["Team"],
        "Pld": 0,
        "W": 0,
        "D": 0,
        "L": 0,
        "GF": 0,
        "GA": 0,
        "GD": 0,
        "Pts": third_place_team["Pts"],
        "Qualification": ""
    }])

    # Concatenate the new row to the existing DataFrame
    third_placed_teams = pd.concat([third_placed_teams.astype(add_third_place.dtypes),
                                    add_third_place.astype(third_placed_teams.dtypes)], axis=0, ignore_index=True)

# Sort third placed teams based on points
third_placed_teams = third_placed_teams.sort_values(by="Pts", ascending=False).reset_index(drop=True)

# Determine which teams advance to knockout phase (top 4)
top_four_third_teams = third_placed_teams.head(4)
top_four_third_teams.loc[:, "Qualification"] = "Advance to knockout phase"

# Display the filled table
top_four_third_teams

In [ ]:
df_fixture_final

In [ ]:
for group in dict_table:
    group_winner = dict_table[group].loc[0, "Team"]
    runners_up = dict_table[group].loc[1, "Team"]
    third_place = dict_table[group].loc[2, "Team"]
    df_fixture_R16.replace({f"Winner {group}": group_winner,
                            f"Runner-up {group}": runners_up,
                            f"3rd {group}": third_place}, inplace=True)

df_fixture_R16["winner"] = "?"
df_fixture_R16

In [ ]:
third_place = dict_table["Group A"].loc[2, "Team"]
third_place

In [ ]:
# Replace "3rd Group D/E/F" with third-place team from Group E
df_fixture_R16["away"].replace("3rd Group D/E/F", dict_table["Group E"].loc[2, "Team"], inplace=True)

# Replace "3rd Group A/D/E/F" with third-place team from Group D
df_fixture_R16["away"].replace("3rd Group A/D/E/F", dict_table["Group D"].loc[2, "Team"], inplace=True)

# Replace "3rd Group A/B/C" with third-place team from Group B
df_fixture_R16["away"].replace("3rd Group A/B/C", dict_table["Group B"].loc[2, "Team"], inplace=True)

# Replace "3rd Group A/B/C/D" with third-place team from Group A
df_fixture_R16["away"].replace("3rd Group A/B/C/D", dict_table["Group A"].loc[2, "Team"], inplace=True)

In [ ]:
df_fixture_R16

In [ ]:
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row["home"], row["away"]
        points_home, points_away = predict_points(home, away)
        
        if points_home > points_away:
            winner = home
        else:
            winner = away

        df_fixture_updated.loc[index, "winner"] = winner
    return df_fixture_updated

In [ ]:
get_winner(df_fixture_R16)

df_fixture_R16

In [ ]:
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, "winner"]
        match = df_fixture_round_1.loc[index, "score"]
        df_fixture_round_2.replace({f"Winner {match}": winner}, inplace=True)
    
    df_fixture_round_2["Winner"] = "?"
    return df_fixture_round_2

In [ ]:
update_table(df_fixture_R16, df_fixture_quarter)

In [ ]:
get_winner(df_fixture_quarter)

In [ ]:
update_table(df_fixture_quarter, df_fixture_semi)

In [ ]:
get_winner(df_fixture_semi)

In [ ]:
update_table(df_fixture_semi, df_fixture_final)

In [ ]:
get_winner(df_fixture_final)